In [1]:
#| default_exp interface_open3d

In [2]:
#| export

from blender_tissue_cartography import io as tcio
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from importlib import reload
from copy import deepcopy

In [4]:
np.set_printoptions(suppress=True)

## Interfacing with `open3d`

In this notebook, we define functions to convert our `ObjMesh` class to and from `open3d`.

Note: `open3d` uses its own array-like datatype, `o3d.core.Tensor` (for autodiff-reasons I think). This means there are also _two_ triangular mesh classes, one `Tensor`-based, one not. We'll convert to the `Tensor`-based one since it appears to be the one which will be supported in the future. 

See https://www.open3d.org/docs/latest/python_api/open3d.t.geometry.TriangleMesh.html

In [12]:
#| export

def convert_to_open3d(mesh: tcio.ObjMesh, add_texture_info=None) -> o3d.t.geometry.TriangleMesh:
    """
    Convert tcio.ObjMesh to open3d.t.geometry.TriangleMesh.
    
    See https://www.open3d.org/docs/latest/python_api/open3d.t.geometry.TriangleMesh.html
    Note: open3d stores its texture coordinates generally as face attributes.
    The returned mesh has a face attribute mesh_o3d.triangle.texture_uvs if 
    texture info is provided. Normals are recomputed by open3d.
    
    Parameters
    ----------
    mesh : tcio.ObjMesh
        Input mesh
    add_texture_info : None or bool
        Whether to add texture info to the pymeshlab.Mesh. If None, texture is added if available
        for all vertices. If True, missing texture info is set to np.nan
    Returns
    -------
    mesh_o3d: o3d.t.geometry.TriangleMesh

    """
    add_texture_info = (not np.isnan(mesh.vertex_textures).any()
                        if add_texture_info is None else add_texture_info)
    
    dtype_f = o3d.core.float32
    dtype_i = o3d.core.int32
    mesh_o3d = o3d.t.geometry.TriangleMesh()
    
    mesh_o3d.triangle.indices = o3d.core.Tensor(mesh.tris, dtype_i)
    mesh_o3d.vertex.positions = o3d.core.Tensor(mesh.vertices, dtype_f)
    if add_texture_info:
        mesh_o3d.triangle.texture_uvs = o3d.core.Tensor(tcio.index_else_nan(mesh.texture_vertices,
                                                                            mesh.texture_tris), dtype_f)
    return mesh_o3d

In [13]:
reload(tcio)

<module 'blender_tissue_cartography.io' from '/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/io.py'>

In [14]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference_preregistered.obj"

In [15]:
mesh_data = tcio.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcio.ObjMesh.read_obj(mesh_fname_ref)

In [17]:
mesh_ref_o3d = convert_to_open3d(mesh_ref, add_texture_info=True)

In [109]:
mesh_ref_o3d.vertex

TensorMap(primary_key="positions") with 1 attribute:
  - positions: shape={56026, 3}, dtype=Float32, device=CPU:0 (primary)
  (Use . to access attributes, e.g., tensor_map.positions)

In [80]:
# this is how you access attributes

np.allclose(mesh_ref_o3d.vertex.positions.numpy(), mesh_ref.vertices)

True

In [81]:
# this is how open3d represents UV maps. Let's create a UV map using the iso-charts algorithm

mesh_ref_o3d.compute_uvatlas()

[Open3D INFO] actual parallel_partitions 1


(0.027833282947540283, 2, 1)

In [82]:
mesh_ref_o3d.triangle.texture_uvs.shape

SizeVector[111104, 3, 2]

In [83]:
# now let's keep the uv info

mesh_ref_o3d = convert_to_open3d(mesh_ref, add_texture_info=True)

In [84]:
mesh_ref_o3d.triangle

TensorMap(primary_key="indices") with 3 attributes:
  - indices    : shape={111104, 3}, dtype=Int32, device=CPU:0 (primary)
  - normals    : shape={111104, 3}, dtype=Float32, device=CPU:0
  - texture_uvs: shape={111104, 3, 2}, dtype=Float32, device=CPU:0
  (Use . to access attributes, e.g., tensor_map.indices)

In [61]:
#| export

def convert_from_open3d(mesh: o3d.t.geometry.TriangleMesh, reconstruct_texture_from_faces=None,
                        texture_vertex_decimals=10) -> tcio.ObjMesh:
    """
    Convert open3d mesh to ObjMesh.
    
    Automatically recomputes vertex normals.
    
    Parameters
    ----------
    mesh : o3d.t.geometry.TriangleMesh
        Input mesh
    reconstruct_texture_from_faces : None or bool
        Whether to reconstruct the texture info
    texture_vertex_decimals : int, default 10
        Decimals UV vertices are rounded to. Needed to merge potential duplicate vertices
        for an economic representation.
    
    Returns
    -------
    tcio.ObjMesh

    """
    if isinstance(mesh, o3d.geometry.TriangleMesh): # legacy ...
        vertices = np.asarray(mesh.vertices)
        mesh.compute_vertex_normals()
        normals = np.asarray(mesh_legacy.vertex_normals)
        normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
        face_matrix = np.asarray(mesh.triangles)
        if reconstruct_texture_from_faces is None:
            reconstruct_texture_from_faces = hasattr(mesh, 'triangle_uvs')
        if not reconstruct_texture_from_faces:
            faces = [[3*[v,] for v in f] for f in face_matrix]
            return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals)
        texture_vertices = np.asarray(mesh_legacy.triangle_uvs).reshape(-1,3,2).reshape(-1, 2, order='F')
        texture_vertices = np.round(texture_vertices, decimals=texture_vertex_decimals)
        texture_vertices_unique, inverse_index = np.unique(texture_vertices, axis=0, return_inverse=True)

        n_faces = face_matrix.shape[0]
        faces = [[[v, inverse_index[ifc+iv*n_faces], v] for iv, v in enumerate(fc)]
                 for ifc, fc in enumerate(face_matrix)]

        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals,
                            texture_vertices=texture_vertices_unique)
    else:    
        vertices = mesh.vertex.positions.numpy()
        mesh.compute_vertex_normals()
        normals = mesh.vertex.normals.numpy()
        normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
        face_matrix = mesh.triangle.indices.numpy()
        if reconstruct_texture_from_faces is None:
            reconstruct_texture_from_faces = "texture_uvs" in mesh.triangle

        if not reconstruct_texture_from_faces:
            faces = [[3*[v,] for v in f] for f in face_matrix]
            return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals)

        # reconstruct texture vertices - big pain.
        texture_vertices = mesh.triangle.texture_uvs.numpy().reshape(-1, 2, order='F')
        texture_vertices = np.round(texture_vertices, decimals=texture_vertex_decimals)
        texture_vertices_unique, inverse_index = np.unique(texture_vertices, axis=0, return_inverse=True)

        n_faces = face_matrix.shape[0]
        faces = [[[v, inverse_index[ifc+iv*n_faces], v] for iv, v in enumerate(fc)]
                 for ifc, fc in enumerate(face_matrix)]

        return tcio.ObjMesh(vertices=vertices, faces=faces, normals=normals,
                            texture_vertices=texture_vertices_unique)

In [62]:
mesh_seams = tcio.ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")

In [63]:
o3d_seams = convert_to_open3d(mesh_seams, add_texture_info=True)

In [64]:
mesh_seams_reconverted = convert_from_open3d(o3d_seams, reconstruct_texture_from_faces=True)

/tmp/ipykernel_1158674/1023165026.py:50: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


In [65]:
convert_from_open3d(o3d_seams.to_legacy())

/tmp/ipykernel_1158674/1023165026.py:29: RuntimeWarning: invalid value encountered in divide
  normals = (normals.T / np.linalg.norm(normals, axis=-1)).T


In [89]:
mesh_seams_reconverted.write_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv_resaved_open3d.obj")